In [1]:
# Load data from Excel file
def GetData(DataFile, DataWorksheet, CasesFile, CasesWorksheet):
    Pallets = LoadFromExcel(DataFile, DataWorksheet, 'Heights')   # List of all pallet sizes (decimetres) that we need to store os shelves
    Pallets.columns = ['Sizes']   # Name the pallets column
    MaxShelves = LoadFromExcel(DataFile, DataWorksheet, 'MaxShelves')   # Maximum number of shelves in a rack
    WeightRacks = LoadFromExcel(DataFile, DataWorksheet, 'WeightRacks')   # Objective function weight on the variable for number of racks
    WeightShelves = LoadFromExcel(DataFile, DataWorksheet, 'WeightShelves')   # Objective function weight on the variable for number of shelves
    ShelfHeights = LoadFromExcel(CasesFile, CasesWorksheet, 'Cases')   # Height of each shelf in each case
    ShelfHeights.columns = [str(n).zfill(1) for n in range(1, ShelfHeights.shape[1] + 1)]   # Label columns from '1' to 'n'
    PalletsPerShelf = LoadFromExcel(DataFile, DataWorksheet, 'PalletsPerShelf')   # Number of pallets on each shelf
    return Pallets, MaxShelves, ShelfHeights, WeightRacks, WeightShelves, PalletsPerShelf

In [ ]:
# Define model data, assigning all data to the Model
def DefineModelData(Model, Case, Pallets, MaxShelves, ShelfHeights, PalletsPerShelf, WeightRacks, WeightShelves):
    Model.MaxShelves = MaxShelves[0].item()
    Model.WeightRacks = WeightRacks[0].item()
    Model.WeightShelves = WeightShelves[0].item()
    Model.PalletsPerShelf = PalletsPerShelf[0].item()
    Model.P = pyo.Set(initialize = range(0, len(Pallets)))
    Model.S = pyo.Set(initialize = range(0, Model.MaxShelves))
    Model.Pallets = pyo.Param(Model.P, within = pyo.NonNegativeIntegers, mutable = True)
    Model.ShelfHeights = pyo.Param(Model.S, within = pyo.NonNegativeIntegers, mutable = True)
    Model.NumShelves = 0

    for p in Model.P:
        Model.Pallets[p] = Pallets['Sizes'][p]
    for s in Model.S:
        Model.ShelfHeights[s] = ShelfHeights[str(s+1)][Case]
        if ShelfHeights[str(s+1)][Case] > 0:
            Model.NumShelves += 1